# Goal is to help Project Manager to create outstanding presentation 🚀
This notebook shows how to use HandleBar Planners with some Plugins to orchestrate process of PowerPoint creation


## 🔥 Let's get the required packages and fire up a kernel

In [44]:
#!import ../config/Settings.cs

#r "nuget: Microsoft.SemanticKernel, 1.3.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.3.1-preview"
#r "nuget: Microsoft.KernelMemory.SemanticKernelPlugin, 0.28.240212.1"

Installed Packages Microsoft.KernelMemory.SemanticKernelPlugin, 0.28.240212.1 Microsoft.SemanticKernel, 1.3.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.3.1-preview

In [45]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Kernel = Microsoft.SemanticKernel.Kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
var builder = Kernel.CreateBuilder();
if(useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey);

var kernel = builder.Build();

## 👨‍💻 Let's load a AI Manager
Here I setup System Description and assist system to behave as a manager in IT company

In [46]:
var systemDescription = @"You are a Project Manager in IT company. You are responsible for the development of a new software product. 
                          You have a team of 5 developers, 2 testers and cloud solution architect. ";

IChatCompletionService chatCompletion = kernel.Services.GetService(typeof(IChatCompletionService)) as IChatCompletionService;
var promptTemplate = new ChatHistory(systemDescription);

In [47]:
promptTemplate.AddUserMessage("Hello");
var result = await chatCompletion.GetChatMessageContentAsync(promptTemplate);

promptTemplate.AddAssistantMessage(result.Content);

## 🔌 Load some plugins
I want to add some plugins to kernel to execute specific actions


In [50]:
var pluginsDirectory = Path.Combine(Directory.GetCurrentDirectory(), "plugins");
var plugin = kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "Manager"));

plugin

Error: System.ArgumentException: An item with the same key has already been added. Key: Manager
   at System.Collections.Generic.Dictionary`2.TryInsert(TKey key, TValue value, InsertionBehavior behavior)
   at System.Collections.Generic.Dictionary`2.Add(TKey key, TValue value)
   at Microsoft.SemanticKernel.KernelPluginCollection.Add(KernelPlugin plugin)
   at Microsoft.SemanticKernel.KernelExtensions.ImportPluginFromPromptDirectory(Kernel kernel, String pluginDirectory, String pluginName, IPromptTemplateFactory promptTemplateFactory)
   at Submission#51.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)